# Sesión 8 - Demostración de la capacidad In-Context Learning de los LLMs para resumen de textos

En este boletín veremos el uso de una técnica llamada **In-Context Learning** mediante la cual los LLMs pueden realizar tareas sin necesidad de ser específicamente entrenados para ellas.

En los últimos años, el enfoque predominante para resolver cualquier tarea de PLN se ha basado en dos etapas:
- **Pre-entrenamiento**: En esta etapa se realiza un entrenamiento auto-supervisado del modelo a partir de enormes cantidades de texto no etiquetado. El pre-entrenamiento no está enfocado a ninguna tarea en particular.

- **Ajuste (Fine-Tuning)**: En esta etapa se realiza un reentrenamiento del modelo pre-entrenado para resolver una tarea concreta utilizando un conjunto de datos, usualmente etiquetados, creado para tal fin.

La creación de conjuntos de datos de entrenamiento para el ajuste fino presenta problemas importantes: no siempre se dispone de suficientes datos anotados y recopilarlos suele llevar mucho tiempo. A medida que aumenta la demanda de aplicaciones de PLN la necesidad de abordar nuevos dominios y tareas se vuelve cada vez más urgente, agravando aún más los problemas que suponen la escasez de conjuntos de datos y la creación de nuevos conjuntos.

Ante este problema, el **In-Context Learning** representa una alternativa: en lugar de realizar un fine-tuning con nuevos datos, los LLMs pueden adaptarse a nuevas tareas simplemente a partir del contexto proporcionado en el prompt. De esya forma, podemos utilizar un mismo modelo para diferentes tareas sin necesidad de realizar un ajuste fino para cada una de ellas.

Hay **tres formas** diferentes de realizar el **In-Context Learning**:

- **Zero Shot Learning**: En el prompt se le indican al modelo las instrucciones para realizar una tarea específica, sin incluir ningún ejemplo de cómo realizarla.
- **Few Shot Learning**: En el prompt se le indican al modelo las instrucciones para resolver una tarea en concreto y, además, se incluyen algunos ejemplos para que el modelo 'entienda' el patrón antes de resolverla.
- **Chain of Thought**: En este caso, además de  instrucciones y ejemplos, en el prompt se incluye el proceso de razonamiento que el modelo debe seguir para resolver la tarea.

En esta sesión:
1. Instalamos las librerias necesarias.
2. Cargaremos el modelo gemma-3-1b-it, lo optimizaremos (cuantizándolo a 4 bits) y lo usaremos sin más ajustes para resolver diferentes tareas de resumen de textos.
3. Visualizaremos la generación progresiva de respuestas con un LLM en tiempo real. Para ello: (1) estructuraremos los mensajes de entrada aplicando una plantilla; y (2) generaremos un relato.
4. Resumiremos textos utilizando las tres posibilidades de in-context-learning: (1) zero-shot, (2) few-shot y (3) chain-of-thought.



Primero, instalaremos las liberías necesarias.

In [ ]:
# Instalamos las librerías necesarias
!pip install transformers huggingface_hub
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

## Apartado 0 - Generación Progresiva de Respuestas con un LLM en Tiempo Real


Antes de explorar las diferentes técnicas de In-Context Learning **vamos a ver cómo se generan las respuestas de un LLM de manera progresiva, mostrando tal respuesta a medida que se va generando**.

Los LLMs pueden generar texto a partir de un prompt. En algunas aplicaciones, tal texto no puede visualizarse hasta que se ha generado por completo. Sin embargo, en otras aplicaciones, como los chatbots, es más apropiado mostrar la respuesta a medida que se genera, proporcionando una experiencia de usuario más dinámica y fluida.


Para la realización de esta práctica vamos a usar un modelo que requiere una licencia de uso. Por ello, es necesario **iniciar una sesion en Hugging Face, usando un token**. **El token que se ha proporcionado es válido hasta el jueves 3 de abril. Para ejecutar el cuaderno después de ese día tendréis que generar un nuevo token.**

Para solicitar una licencia de uso y crear un token hay que acceder a la página del modelo en Hugging Face. **El proceso es bastante sencillo y el uso de este modelo es gratuito.** Podéis encontrar una descripción detallada en: https://huggingface.co/docs/hub/security-tokens



In [ ]:
# Importamos las librerias necesarias
from huggingface_hub import login

# Este token solo va a funcionar hoy, jueves 3 de abril, para poder ejecutar el cuaderno después de ese día teneis que incluir vuestro propio token.
login(token="")

En esta práctica vamos a usar el modelo **gemma-3-1b-it**, que es una variante de la serie Gemma 3, desarrollada por Google.

**gemma-3-1b-it** tiene aproximadamente 1.000 millones (`1b`) de paramétros y ha sido entrenado para seguir instrucciones específicas, lo que lo hace ideal para tareas que requieren comprensión de comandos y generan respuestas basadas en ellos (it).

Ahora, cargamos el modelo:

In [ ]:
# Importamos las librerias necesarias
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM

# Definimos el modelo que vamos a usar
model_path = "google/gemma-3-1b-it"

# Configuración la cuantización de 4-bit
quantization_config = BitsAndBytesConfig(load_in_4bit = True)

# Cargamos el tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Cargamos el modelo
# Estamos cuantizando el modelo a 4-bits para que ocupe menos espacio en GPU
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config = quantization_config,
    device_map = "auto"
).eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Para interactuar con el modelo Gemma, necesitamos estructurar los mensajes de entrada en un formato específico para que el modelo pueda comprender lo que le estamos pidiendo.

En este caso, configuramos un mensaje que simula una conversación, donde el usuario pide al modelo que genere una historia en la que un perro y un gato se pelean por una albóndiga.

In [ ]:
# Estructuramos los mensajes de entrada en el formato requerido por Gemma
messages = [
  {
     "role": "user",
     "content": "Escribe una historia en la que un perro y un gato se pelean \
                 por una albóndiga. Quiero que la historia sea muy graciosa.",
  },
]

# Aplicamos un template de chat al mensaje de entrada utilizando el tokenizador.
# Esto formatea los mensajes según el formato esperado por el modelo,
# añadiendo un prompt
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    tokenize = True,
    return_tensors = "pt",
    return_dict = True,
).to(model.device)

Ahora, vamos a generar el cuento:

In [ ]:
# Importamos las librerias necesarias
from transformers import TextIteratorStreamer
from threading import Thread

# Inicializamos un TextIteratorStreamer, que permite generar texto en tiempo real
streamer = TextIteratorStreamer(
    tokenizer,
    skip_prompt = True,  # Omite el prompt en la salida
    skip_special_tokens = True # Omite tokens especiales
)

# Configuramos los parámetros para la generación
generation_kwargs = dict(

    **inputs,

    # Limitamos el máximo de tokens que el modelo puede responder
    max_new_tokens=1000,

    # Usamos el streamer para manejar la salida generada
    streamer = streamer,

    # Habilita el muestreo, lo que permite ajustar la aleatoriedad en la generación
    do_sample=True,

    # Temperatura: controla el grado de aleatoriedad en la generación
    temperature=0.7,

    # top-p: umbral para seleccionar las palabras candidatas para la siguiente palabra
    top_p = 0.9,

    # top-k: número de palabras candidatas a ser la siguiente palabra generada
    top_k = 50
)

# Configuramos un hilo para no bloquear el notebook
thread = Thread(target = model.generate, kwargs = generation_kwargs)

# Iniciamos el hilo
thread.start()

# Mostramos la respuesta del modelo
print("Respuesta del modelo:")
for token in streamer:
  print(token, end="", flush = True)  # `flush=True` para ver la salida inmediatamente

# Esperar a que termine la generación
thread.join()

Respuesta del modelo:
El sol caía sobre la pequeña ciudad de Hollow 

KeyboardInterrupt: 

## Apartado 1 - Resumen de textos con LLMs mediante In-Context Learning

Vamos a seguir usando el modelo gemma-3-1b-it.

Vamos a definir una función que realice el resumen de un par de textos. Esta función nos permite modificar tanto el prompt a utilizar como los parámetros para el muestro: temperatura, top-p y top-k.

Esta función nos permitirá tener un codigo más limpio.


In [ ]:
def generate_response(base_prompt,
                      texts,
                      model = model,
                      tokenizer = tokenizer,
                      temp = 0.9,
                      t_p = 0.95,
                      t_k = 35):

  responses = []

  # Para cada texto a resumir
  for text in texts:

    # Estructuramos los mensajes de entrada en el formato requerido por Gemma
    messages = [
        {
            "role": "user",
            "content": base_prompt + " " + text,
        },
    ]

    # Aplicamos un template de chat al mensaje de entrada utilizando el tokenizador.
    # Esto formatea los mensajes según el formato esperado por el modelo, añadiendo un prompt
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True,
        tokenize = True,
        return_tensors ="pt",
        return_dict = True,
    ).to(model.device)

    # Generamos la respuesta del modelo
    outputs = model.generate(**inputs,
                            max_new_tokens = 1000,
                            temperature = temp,
                            top_k = t_k,
                            top_p = t_p,
                            do_sample = True
                            )

    # Obtenemos la respuesta generada
    # Realizo un poco de postprocesamiento para obtener solo el nuevo texto generado
    response = "".join(tokenizer.decode(outputs[0], skip_special_tokens=True).split("model\n")[-1].split("\n")[:-1])

    # Mostramos la respuesta
    print("Texto a resumir:")
    print(text)
    print("Respuesta del modelo:")
    print(response)
    print()

    responses.append(response)

  return responses

### Apartado 1.1 - Zero-Shot Learning


Comenzamos explorando **la estrategia más sencilla** de In-Context Learning: el **Zero-Shot Learning**.

En esta estrategia, usamos un modelo para resolver una tarea sin proporcionarle ejemplos previos de cómo hacerlo: simplemente le damos una lista de instrucciones en el prompt, y el modelo intenta generar una respuesta basándose en el prompt y en el conocimiento adquirido durante su preentrenamiento.

El flujo general consiste en los siguientes pasos:

1.   Cargar el modelo y su tokenizador.
2.   Tokenizar el texto y procesarlo con el modelo: convertimos el texto en una representación numérica que el modelo pueda interpretar.
3.   Predicción: aplicamos la función softmax para calcular la probabilidad de pertenencia del texto a cada una de las etiquetas.
4.   Evaluación: calculamos la probabilidad de que la premisa (texto) implique la hipótesis (cada una de las etiquetas), lo que nos permitirá clasificar correctamente el texto.

Definimos el prompt y los textos a resumir:

In [ ]:
zero_shot_prompt = """
Eres un modelo cuya tarea es resumir el texto recibido en una sola oración.

Por favor, resume el siguiente texto:"""


full_texts = [
"El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia \
artificial que permite a las máquinas entender, interpretar y generar texto de \
manera similar a como lo hacen los seres humanos. Este campo se aplica en muchas \
áreas, como chatbots, traducción automática y análisis de sentimientos.",

"Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos \
textuales para identificar patrones y relaciones entre palabras y frases. \
Esto permite que las máquinas realicen tareas como responder preguntas, \
generar texto o identificar la intención detrás de un mensaje.",

"La tokenización es uno de los primeros pasos en el PLN, donde un texto se \
divide en partes más pequeñas llamadas tokens. Estos tokens pueden ser \
palabras, subpalabras o incluso caracteres, y se utilizan para facilitar \
el procesamiento y la comprensión del lenguaje por parte de los modelos \
de aprendizaje automático."

"Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos \
textuales para identificar patrones y relaciones entre palabras y frases. \
Esto permite que las máquinas realicen tareas como responder preguntas, \
generar texto o identificar la intención detrás de un mensaje.",

"La tokenización es uno de los primeros pasos en el PLN, donde un texto se \
divide en partes más pequeñas llamadas tokens. Estos tokens pueden ser \
palabras, subpalabras o incluso caracteres, y se utilizan para facilitar \
el procesamiento y la comprensión del lenguaje por parte de los modelos \
de aprendizaje automático."
]

Procedemos a usar el modelo para resumir cada uno de los textos:

In [ ]:
resumenes_zero = generate_response(zero_shot_prompt, full_texts)

Texto a resumir:
El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas entender, interpretar y generar texto de manera similar a como lo hacen los seres humanos. Este campo se aplica en muchas áreas, como chatbots, traducción automática y análisis de sentimientos.
Respuesta del modelo:
El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que busca que las máquinas comprendan y respondan al lenguaje humano.

Texto a resumir:
Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para identificar patrones y relaciones entre palabras y frases. Esto permite que las máquinas realicen tareas como responder preguntas, generar texto o identificar la intención detrás de un mensaje.
Respuesta del modelo:
Los modelos de lenguaje en Procesamiento del Lenguaje Natural (PLN) se aprenden con grandes cantidades de texto para reconocer patrones y relaciones, lo que les permite a las

### Apartado 1.2 - Few-Shot Learning


Usaremos ahora **Few-Shot Learning**. En esta estrategia, proporcionamos al modelo algunos ejemplos para resolver la tarea.

Estos ejemplos se incluyen en el prompt para guiar al modelo en la tarea específica. Gracias a estos ejemplos, el modelo tiene acceso a muestras que le permiten entender mejor el patrón de la tarea y generar una respuesta más precisa.

Al igual que antes, el flujo general consiste en los siguientes pasos:

1.   Cargar el modelo de clasificación y su tokenizador.
2.   Tokenizar el texto y procesarlo con el modelo.
3.   Predicción.
4.   Evaluación.

In [ ]:
few_shot_prompt = """
Eres un modelo cuya tarea es resumir el texto recibido en una sola oración.

Ejemplos:
Texto: El análisis de sentimientos es una técnica del PLN que permite identificar las emociones o actitudes expresadas en un texto. Se utiliza ampliamente en redes sociales, encuestas y reseñas de productos para comprender las opiniones y el estado de ánimo de los usuarios. Los modelos de análisis de sentimientos pueden clasificar el texto como positivo, negativo o neutral.
Resumen: El análisis de sentimientos identifica emociones en el texto, clasificado como positivo, negativo o neutral, y se usa en redes sociales y reseñas de productos.
Texto: La traducción automática ha avanzado mucho gracias al uso de modelos neuronales que pueden aprender a traducir entre diferentes idiomas sin necesidad de reglas explícitas. Sin embargo, aún existen desafíos para traducir correctamente expresiones idiomáticas y textos con significados complejos.
Resumen: La traducción automática ha mejorado con modelos neuronales, pero sigue siendo un reto traducir expresiones idiomáticas.

No incluyas ninguno de los ejemplos vistos en el resumen del texto de entrada.
Ahora, por favor, resume el siguiente texto:"""


Procedemos a usar el modelo para resumir cada uno de los textos:

In [ ]:
resumenes_few = generate_response(few_shot_prompt, full_texts)

Texto a resumir:
El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas entender, interpretar y generar texto de manera similar a como lo hacen los seres humanos. Este campo se aplica en muchas áreas, como chatbots, traducción automática y análisis de sentimientos.
Respuesta del modelo:
El Procesamiento del Lenguaje Natural (PLN) es el campo de la IA que permite a las máquinas comprender y responder al lenguaje humano, y que tiene aplicaciones en chatbots, traducción automática y análisis de sentimientos.

Texto a resumir:
Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para identificar patrones y relaciones entre palabras y frases. Esto permite que las máquinas realicen tareas como responder preguntas, generar texto o identificar la intención detrás de un mensaje.
Respuesta del modelo:


Texto a resumir:
La tokenización es uno de los primeros pasos en el PLN, donde un texto se divide en part

### Apartado 1.3 - Chain of Thought

Usamos ahora el enfoque **Chain of Thought**.

En esta estrategia, **el prompt no solo incluye ejemplos**, sino **también el razonamiento detallado** que el modelo debe seguir para llegar a la respuesta correcta.

Al estructurar el prompt de esta manera, guiamos al modelo en la resolución de la tarea, permitiéndole identificar patrones y mejorar la precisión de sus respuestas.

Al igual que antes, el flujo general consiste en:

1.   Cargar el modelo de clasificación y su tokenizador.
2.   Tokenizar el texto y procesarlo con el modelo.
3.   Predicción.
4.   Evaluación.

In [ ]:
cot_prompt = """
Eres un modelo cuya tarea es resumir el texto recibido en una sola oración.
Primero, analiza el texto detenidamente y proporciona un análisis del contenido, destacando los puntos clave que deben ser incluidos en el resumen. Luego, resume el texto en una sola oración, manteniendo la coherencia y abarcando los puntos más importantes.
Finalmente, da la respuesta en el siguiente formato:

Resumen: [resumen del texto de entrada]

Ejemplos:
Texto: El análisis de sentimientos es una técnica del PLN que permite identificar las emociones o actitudes expresadas en un texto. Se utiliza ampliamente en redes sociales, encuestas y reseñas de productos para comprender las opiniones y el estado de ánimo de los usuarios. Los modelos de análisis de sentimientos pueden clasificar el texto como positivo, negativo o neutral.
Resumen: El análisis de sentimientos identifica emociones en el texto, clasificado como positivo, negativo o neutral, y se usa en redes sociales y reseñas de productos.
Texto: La traducción automática ha avanzado mucho gracias al uso de modelos neuronales que pueden aprender a traducir entre diferentes idiomas sin necesidad de reglas explícitas. Sin embargo, aún existen desafíos para traducir correctamente expresiones idiomáticas y textos con significados complejos.
Resumen: La traducción automática ha mejorado con modelos neuronales, pero sigue siendo un reto traducir expresiones idiomáticas.

Ahora, por favor, resume el siguiente texto:"""


Procedemos a usar el modelo para resumir cada uno de los textos:

In [ ]:
resumenes_cot = generate_response(cot_prompt, full_texts)

Texto a resumir:
El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas entender, interpretar y generar texto de manera similar a como lo hacen los seres humanos. Este campo se aplica en muchas áreas, como chatbots, traducción automática y análisis de sentimientos.
Respuesta del modelo:
Resumen: El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas entender y procesar lenguaje humano, y tiene aplicaciones diversas como chatbots, traducción automática y análisis de sentimiento.

Texto a resumir:
Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para identificar patrones y relaciones entre palabras y frases. Esto permite que las máquinas realicen tareas como responder preguntas, generar texto o identificar la intención detrás de un mensaje.
Respuesta del modelo:
Resumen: Los modelos de lenguaje en Procesamiento del Lenguaje Natural (PLN) 

Vamos a ver los resumenes obtenidos con cada una de las técnicas:

In [ ]:
for idx in range(len(resumenes_zero)):
  print(resumenes_zero[idx])
  print(resumenes_few[idx])
  print(resumenes_cot[idx])
  print()

El Procesamiento del Lenguaje Natural (PLN) es el campo de la inteligencia artificial que permite a las máquinas comprender y responder al lenguaje humano.
El Procesamiento del Lenguaje Natural (PLN) es la rama de la IA que permite a las máquinas comprender y procesar el lenguaje humano, abarcando áreas como chatbots, traducción automática y análisis de sentimientos.
Resumen: El Procesamiento del Lenguaje Natural (PLN) es el campo de la IA que permite a las máquinas comprender y generar lenguaje humano, abarcando aplicaciones como chatbots, traducción automática y análisis de sentimientos.

Los modelos de lenguaje en Procesamiento del Lenguaje Natural (PLN) se entrenan con grandes cantidades de texto para aprender patrones y relaciones en el lenguaje, lo que les permite realizar tareas como responder preguntas, generar texto y entender la intención de un mensaje.
Los modelos de lenguaje en PLN aprenden de grandes cantidades de texto para entender patrones y relaciones, lo que les permi

## Apartado 2 - La aleatoriedad en las respuestas

Los LLMs **no son deterministas**, es decir, no siempre producen la misma respuesta para una misma entrada.

Esto se debe a que la **generación de texto incorpora un cierto grado de aleatoriedad** que permite que las respuestas sean más variadas y naturales, en lugar de rígidas y repetitivas.

Para **controlar este comportamiento**, existen una serie de parámetros que le indican al modelo como realizar la decodificación (generación de la siguiente palabra):
```
  # Generamos la respuesta del modelo
  outputs = model.generate(inputs.input_ids,
                           max_length=50,
                           temperature=0.9,
                           top_k=35,
                           top_p=0.95,
                           do_sample=True
                           )
```

- **Temperatura**: que controla lo aleatorio que es el texto generado dando más peso (t>1) o menos peso (t<1) a las palabras menos probables.
- **Top-p**: que elige las palabras más probables hasta que su probabilidad acumulada supere un umbral (p).
- **Top-k**: que limita la cantidad de palabras candidatas que el modelo puede elegir en cada paso a las k más ptobables.

Vamos a definir una función que realice el resumen de un par de textos. Esta función nos permite varias los valores de temperatura, top-p y top-k.

Esta función nos permitirá tener un codigo más limpio.


In [ ]:
# Definimos el prompt
few_shot_prompt = """
Eres un modelo cuya tarea es resumir el texto recibido en una sola oración.

Ejemplos:
Texto: El análisis de sentimientos es una técnica del PLN que permite identificar las emociones o actitudes expresadas en un texto. Se utiliza ampliamente en redes sociales, encuestas y reseñas de productos para comprender las opiniones y el estado de ánimo de los usuarios. Los modelos de análisis de sentimientos pueden clasificar el texto como positivo, negativo o neutral.
Resumen: El análisis de sentimientos identifica emociones en el texto, clasificado como positivo, negativo o neutral, y se usa en redes sociales y reseñas de productos.
Texto: La traducción automática ha avanzado mucho gracias al uso de modelos neuronales que pueden aprender a traducir entre diferentes idiomas sin necesidad de reglas explícitas. Sin embargo, aún existen desafíos para traducir correctamente expresiones idiomáticas y textos con significados complejos.
Resumen: La traducción automática ha mejorado con modelos neuronales, pero sigue siendo un reto traducir expresiones idiomáticas.

No incluyas ninguno de los ejemplos vistos en el resumen del texto de entrada.
Ahora, por favor, resume el siguiente texto:
"""


Vamos a ver como se comporta el modelo si variamos estos parámetros:

### Generación determinista (con la mínima aleatoriedad posible)

En esta configuración, el modelo siempre elegirá la palabra más probable en cada paso (greedy decoding), sin ningún tipo de variación.

Estableciendo `top_k=0` y `top_p=0`, estamos deshabilitando estas opciones para el muestreo.

Al fijar `temperature=0.01`, el modelo elimina al máximo la aleatoriedad (prácticamente greedy decoding) y elegirá la palabra más probable en cada momento. Como consecuencia, el modelo generará la misma respuesta o una respuesta muy parecida cada vez que se ejecute sobre una misma entrada. Es importante tener en cuenta que no es posible eliminar la aleatoriedad al completo.

Como resultado, las respuestas son consistentes y predecibles, pero también pueden volverse monótonas y poco creativas.

Este tipo de configuración es útil en tareas donde la precisión es más importante que la diversidad, como en resúmenes técnicos o extracción de entidades.

In [ ]:
generate_response(few_shot_prompt, full_texts, temp = 0.01, t_p = 0, t_k = 0)

Texto a resumir:
El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas entender, interpretar y generar texto de manera similar a como lo hacen los seres humanos. Este campo se aplica en muchas áreas, como chatbots, traducción automática y análisis de sentimientos.
Respuesta del modelo:
El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas comprender y generar texto, y se utiliza en chatbots, traducción automática y análisis de sentimientos.

Texto a resumir:
Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para identificar patrones y relaciones entre palabras y frases. Esto permite que las máquinas realicen tareas como responder preguntas, generar texto o identificar la intención detrás de un mensaje.
Respuesta del modelo:
Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para aprender patrones y rel

['El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas comprender y generar texto, y se utiliza en chatbots, traducción automática y análisis de sentimientos.',
 'Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para aprender patrones y relaciones entre palabras y frases, lo que les permite realizar tareas como responder preguntas, generar texto y entender la intención de un mensaje.',
 'La tokenización es un proceso fundamental en el Procesamiento del Lenguaje Natural (PLN) que divide textos en unidades más pequeñas (tokens) para facilitar el análisis y la comprensión del lenguaje por parte de los modelos de aprendizaje automático.']

In [ ]:
generate_response(few_shot_prompt, full_texts, temp = 0.01, t_p = 0, t_k = 0)

Texto a resumir:
El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas entender, interpretar y generar texto de manera similar a como lo hacen los seres humanos. Este campo se aplica en muchas áreas, como chatbots, traducción automática y análisis de sentimientos.
Respuesta del modelo:
El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas comprender y generar texto, y se utiliza en chatbots, traducción automática y análisis de sentimientos.

Texto a resumir:
Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para identificar patrones y relaciones entre palabras y frases. Esto permite que las máquinas realicen tareas como responder preguntas, generar texto o identificar la intención detrás de un mensaje.
Respuesta del modelo:
Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para aprender patrones y rel

['El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas comprender y generar texto, y se utiliza en chatbots, traducción automática y análisis de sentimientos.',
 'Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para aprender patrones y relaciones entre palabras y frases, lo que les permite realizar tareas como responder preguntas, generar texto y entender la intención de un mensaje.',
 'La tokenización es un proceso fundamental en el Procesamiento del Lenguaje Natural (PLN) que divide textos en unidades más pequeñas (tokens) para facilitar el análisis y la comprensión del lenguaje por parte de los modelos de aprendizaje automático.']

Podemos observar que con la configuración escogida (`temperature = 0.01`, `top_p = 0`, `top_k = 0`), las diferentes llamadas a la función generate_response producen los mismos resúmenes.

Esto es justo lo esperado, ya que esta configuración reduce al mínimo la aleatoreidad del modelo, que responde así de la forma más determinista posible, eligiendo las palabras más probables en cada paso (greedy decoding) y generando respuestas idénticas o muy similares en cada ejecución.

Sin embargo, aún reduciendo al mínimo la aleatoriedad, es posible que algunas respuestas varíen ligeramente debido a factores internos del proceso de generación.

### Generación con aletoriedad controlada (equilibrio entre coherencia y creatividad)

En esta configuración, la temperatura moderada y el muestreo restringido hacen que el modelo tenga cierta variabilidad en sus respuestas sin perder coherencia.

Para ello, limitamos la selección de palabras a un conjunto de opciones más probables mediante `top_k=50` y `top_p=0.9`. Por otro lado, fijamos  `temperature=0.7`, con lo que se introduce un grado controlado de aleatoriedad que evita respuestas excesivamente rígidas sin hacerlas impredecibles o incoherentes.

Como resultado, el texto generado mantiene un equilibrio entre diversidad y precisión.

Esta configuración es ideal para tareas como la generación de texto creativo o el uso en chatbots.

In [ ]:
generate_response(few_shot_prompt, full_texts, temp = 0.7, t_p = 0.9, t_k = 50)

Texto a resumir:
El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas entender, interpretar y generar texto de manera similar a como lo hacen los seres humanos. Este campo se aplica en muchas áreas, como chatbots, traducción automática y análisis de sentimientos.
Respuesta del modelo:
El Procesamiento del Lenguaje Natural (PLN) es una rama de la IA que permite a las máquinas comprender y procesar el lenguaje humano, abarcando áreas como chatbots, traducción automática y análisis de sentimientos.

Texto a resumir:
Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para identificar patrones y relaciones entre palabras y frases. Esto permite que las máquinas realicen tareas como responder preguntas, generar texto o identificar la intención detrás de un mensaje.
Respuesta del modelo:
Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para aprender patrones y relac

['El Procesamiento del Lenguaje Natural (PLN) es una rama de la IA que permite a las máquinas comprender y procesar el lenguaje humano, abarcando áreas como chatbots, traducción automática y análisis de sentimientos.',
 'Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para aprender patrones y relaciones, lo que permite realizar tareas como responder preguntas, generar texto y entender la intención de un mensaje.',
 'La tokenización es un paso fundamental en el Procesamiento del Lenguaje Natural (PLN), donde el texto se divide en unidades más pequeñas (tokens) para facilitar el análisis y la comprensión del lenguaje por parte de los modelos de aprendizaje automático.']

In [ ]:
generate_response(few_shot_prompt, full_texts, temp = 0.7, t_p = 0.9, t_k = 50)

Texto a resumir:
El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas entender, interpretar y generar texto de manera similar a como lo hacen los seres humanos. Este campo se aplica en muchas áreas, como chatbots, traducción automática y análisis de sentimientos.
Respuesta del modelo:
El Procesamiento del Lenguaje Natural (PLN) es el campo de la IA que permite a las máquinas entender y procesar el lenguaje humano, y se utiliza en diversas aplicaciones como chatbots, traducción automática y análisis de sentimientos.

Texto a resumir:
Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para identificar patrones y relaciones entre palabras y frases. Esto permite que las máquinas realicen tareas como responder preguntas, generar texto o identificar la intención detrás de un mensaje.
Respuesta del modelo:
Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para apren

['El Procesamiento del Lenguaje Natural (PLN) es el campo de la IA que permite a las máquinas entender y procesar el lenguaje humano, y se utiliza en diversas aplicaciones como chatbots, traducción automática y análisis de sentimientos.',
 'Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para aprender patrones y relaciones entre palabras y frases, lo que permite la generación de texto, la respuesta a preguntas y la identificación de la intención.',
 '']

Con la configuración escogida (`temperature = 0.7`, `top_p = 0.9`, `top_k = 50`), las diferentes llamadas a gnerate_resumen producen resultados  similares, pero no idénticos.

Con esta configuración el modelo mantiene un grado de aleatoriedad que produce respuestas diferentes en cada ejecución (diversidad), pero siempre dentro de un rango coherente (calidad).

Es importante recordar que, aunque la variabilidad es mayor que en configuraciones más deterministas, el modelo sigue generando respuestas dentro de un conjunto de opciones probables, por lo que las diferencias no suelen ser muy grandes.

### Generación creativa (mayor aleatoriedad)

En esta configuración, la temperatura alta y el muestreo sin restricciones hacen que el modelo explore opciones más inusuales en la generación de texto.

Al establecer `temperature=1.2`, se introduce un alto grado de aleatoriedad, lo que fomenta respuestas más diversas y menos predecibles. Además, con `top_p=1.0` y `top_k=0`, no se restringe la selección de palabras, permitiendo que el modelo elija entre un rango amplio de opciones posibles.

Como resultado, el texto generado puede ser más creativo e inesperado, aunque también aumenta el riesgo de incoherencias.

Esta configuración es útil en tareas donde se busca diversidad, como la generación de textos literarios o el brainstorming a partir de una idea.

In [ ]:
generate_response(few_shot_prompt, temperature = 1.2, top_p = 1.0, top_k = 0)

Texto a resumir:
El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas entender, interpretar y generar texto de manera similar a como lo hacen los seres humanos. Este campo se aplica en muchas áreas, como chatbots, traducción automática y análisis de sentimientos.
Respuesta del modelo:
El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas comprender, interpretar y generar texto similar al humano, y se aplica en áreas como chatbots, traducción automática y análisis de sentimientos.

Texto a resumir:
Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para identificar patrones y relaciones entre palabras y frases. Esto permite que las máquinas realicen tareas como responder preguntas, generar texto o identificar la intención detrás de un mensaje.
Respuesta del modelo:
Los modelos de lenguaje en PLN se entrenan con grandes cantidades de dat

['El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas comprender, interpretar y generar texto similar al humano, y se aplica en áreas como chatbots, traducción automática y análisis de sentimientos.',
 'Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para aprender patrones y relaciones entre palabras y frases, lo que les permite realizar tareas como responder preguntas, generar texto y entender la intención de un mensaje.',
 '']

In [ ]:
generate_response(few_shot_prompt, temperature = 1.2, top_p = 1.0, top_k = 0)

Texto a resumir:
El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas entender, interpretar y generar texto de manera similar a como lo hacen los seres humanos. Este campo se aplica en muchas áreas, como chatbots, traducción automática y análisis de sentimientos.
Respuesta del modelo:
El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas entender y procesar el lenguaje humano, y se aplica en diversas áreas como chatbots, traducción automática y análisis de sentimientos.

Texto a resumir:
Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para identificar patrones y relaciones entre palabras y frases. Esto permite que las máquinas realicen tareas como responder preguntas, generar texto o identificar la intención detrás de un mensaje.
Respuesta del modelo:
Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textual

['El Procesamiento del Lenguaje Natural (PLN) es un campo de la inteligencia artificial que permite a las máquinas entender y procesar el lenguaje humano, y se aplica en diversas áreas como chatbots, traducción automática y análisis de sentimientos.',
 'Los modelos de lenguaje en PLN se entrenan con grandes cantidades de datos textuales para aprender patrones y relaciones, lo que les permite realizar tareas como responder preguntas, generar texto y entender la intención de un mensaje.',
 '']

Con la configuración escogida (`temperature = 1.2`, `top_p = 1.0`, `top_k = 0`), las diferentes llamadas a generate_resumen producen resultados notablemente diferentes.

Con esta configuración, el modelo tiene un alto grado de aleatoriedad en la generación de texto que incrementa la creatividad de las respuestas, pero también puede generar respuestas incoherentes e inesperadas.

En resumen, debido al alto grado de aleatoriedad en la configuración, los resultados tienden a ser muy diferentes entre sí en cada ejecución, lo que puede ser útil cuando se busca diversidad, pero también puede resultar en respuestas menos coherentes.